In [25]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=0)
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [11]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_Fs2_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_Fs2_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_Fs2_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs2_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [12]:
dataset = pd.read_csv('datasets/Fs3_B_O_DANCE_WALK_KIN_0.5sec.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_neck_chest_distance,avg_neck_rotation_w,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.684605,0.268051,0.236534,0.478955,0.469381,0.455152,0.351238,0.336392,0.278741,0.011497,...,0.531111,0.677884,1.109843,0.766334,1.537713,1.062221,1.355767,-0.5,0.6,0.9
1,0.438911,0.265254,0.229588,0.283025,0.468163,0.452005,0.350459,0.333469,0.278750,0.011885,...,0.014436,0.021416,1.085836,0.779991,1.575538,1.049393,1.367237,-0.5,0.6,0.9
2,0.440199,0.266441,0.223630,0.309053,0.455287,0.439872,0.343657,0.325378,0.278739,0.017740,...,0.072819,0.191442,1.231576,1.117785,0.133194,0.126177,0.355767,-0.5,0.6,0.9
3,0.500120,0.386881,0.393727,0.452718,0.418828,0.440395,0.324313,0.340373,0.278729,0.033718,...,0.464108,0.605356,1.068333,1.038694,1.638293,1.021144,1.332992,-0.5,0.6,0.9
4,0.562528,0.447009,0.596923,0.488571,0.361697,0.522971,0.286162,0.430045,0.278747,0.041425,...,0.139125,0.207208,1.304347,1.077684,1.563165,0.917321,0.803440,-0.5,0.6,0.9


In [13]:
train_dataset = dataset.sample(frac=0.95, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 38538
No Test Samples: 2028


In [14]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [15]:
train_emotions_p = model_p.predict(train_dataset)
train_emotions_a = model_a.predict(train_dataset)
train_emotions_d = model_d.predict(train_dataset)

rows = []
for i in range(len(train_dataset)):
    rows.append([train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]])

train_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,-0.224623,0.252498,0.183831
1,0.136480,0.351693,0.042269
2,0.381650,0.514686,0.338079
3,-0.181635,0.254569,0.151412
4,0.244220,0.508577,0.134054


In [16]:
test_emotions_p = model_p.predict(test_dataset)
test_emotions_a = model_a.predict(test_dataset)
test_emotions_d = model_d.predict(test_dataset)

rows = []
for i in range(len(test_dataset)):
    rows.append([test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]])

test_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.342541,0.571836,0.268902
1,0.202149,0.474657,0.209683
2,-0.044190,0.513493,0.240185
3,0.145442,0.547945,0.058469
4,0.194220,0.576303,0.171302


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [55]:
def baseline_model():
    model = keras.Sequential()
    #model.add(layers.Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
    model.add(layers.Normalization())
    model.add(layers.Dense(8,kernel_initializer='normal', activation='relu'))
    model.add(layers.Dense(16,kernel_initializer='normal', activation='relu'))
    model.add(layers.Dense(26,kernel_initializer='normal', activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

In [56]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, train_emotions, train_dataset, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


Baseline: -0.12 (0.00) MSE


In [57]:
estimator.fit(train_emotions, train_dataset)

In [59]:
generated = estimator.predict(test_emotions)

print(generated)

[[0.73803294 0.4104459  0.40309125 ... 1.77951    1.9179385  1.2821218 ]
 [0.55641913 0.34994015 0.3481233  ... 1.0547447  1.1152802  0.90968597]
 [0.4383375  0.31570366 0.36830238 ... 1.0532228  0.8752621  0.849954  ]
 ...
 [0.5564265  0.37400472 0.40802303 ... 0.79345167 1.0101264  1.03623   ]
 [0.58065    0.34447706 0.325531   ... 0.5543004  0.8214466  0.5117859 ]
 [0.6328031  0.37856296 0.4033006  ... 1.1373792  1.2788883  1.2928975 ]]


In [60]:
mae_errors = mean_absolute_error(test_dataset, generated, multioutput='raw_values')
mse_errors = mean_squared_error(test_dataset, generated, multioutput='raw_values')

features = ["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          #"avg_neck_chest_distance",
          "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
          "avg_total_body_volume",
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
         ]

print("Overall MAE: " + str(mean_absolute_error(test_dataset, generated)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print("Example [Regen-Real]: " + str(generated[i][i]) + " - " + str(test_dataset.iloc[i,i]))
    print()

Overall MAE: 0.18178509602378795

==max_hand_distance==
MSE: 0.05759
MAE: 0.19167
Example [Regen-Real]: 0.73803294 - 0.5014314150175755

==avg_l_hand_hip_distance==
MSE: 0.01821
MAE: 0.10312
Example [Regen-Real]: 0.34994015 - 0.4635218581442564

==avg_r_hand_hip_distance==
MSE: 0.02231
MAE: 0.11525
Example [Regen-Real]: 0.36830238 - 0.1284764405043354

==max_stride_length==
MSE: 0.02082
MAE: 0.11231
Example [Regen-Real]: 0.41476217 - 0.1824674140137199

==avg_l_hand_chest_distance==
MSE: 0.00974
MAE: 0.07535
Example [Regen-Real]: 0.38939866 - 0.319366799465508

==avg_r_hand_chest_distance==
MSE: 0.01087
MAE: 0.08042
Example [Regen-Real]: 0.44297132 - 0.4607566991241786

==avg_l_elbow_hip_distance==
MSE: 0.00465
MAE: 0.04965
Example [Regen-Real]: 0.2885767 - 0.3329558286749665

==avg_r_elbow_hip_distance==
MSE: 0.00529
MAE: 0.05407
Example [Regen-Real]: 0.34560227 - 0.3483186167462977

==avg_chest_pelvis_distance==
MSE: 0.00001
MAE: 0.00223
Example [Regen-Real]: 0.27428284 - 0.277365265

In [61]:
gen_emotions_p = model_p.predict(generated)
gen_emotions_a = model_a.predict(generated)
gen_emotions_d = model_d.predict(generated)

rows = []
for i in range(len(test_dataset)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.153910,0.555116,0.087140
1,0.214776,0.476516,0.224074
2,0.068179,0.520809,0.107089
3,0.150822,0.579803,0.124467
4,0.111411,0.544633,0.166775


In [62]:
test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.342541,0.571836,0.268902
1,0.202149,0.474657,0.209683
2,-0.044190,0.513493,0.240185
3,0.145442,0.547945,0.058469
4,0.194220,0.576303,0.171302


In [63]:
mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_dataset, generated)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(10):
    print("Real: " + str([test_emotions.iloc[i,0], test_emotions.iloc[i,1], test_emotions.iloc[i,2]]))
    print("Generated: " + str([gen_emotions.iloc[i,0], gen_emotions.iloc[i,1], gen_emotions.iloc[i,2]]))
    print()

Overall MAE: 0.18178509602378795

==PLEASURE==
MSE: 0.00586
MAE: 0.06128

==AROUSAL==
MSE: 0.00144
MAE: 0.02960

==DOMINANCE==
MSE: 0.00391
MAE: 0.04772

Real: [0.3425409, 0.57183623, 0.26890206]
Generated: [0.1539098, 0.55511636, 0.08713962]

Real: [0.20214874, 0.4746569, 0.20968264]
Generated: [0.2147757, 0.47651622, 0.22407399]

Real: [-0.044190075, 0.51349324, 0.24018456]
Generated: [0.06817896, 0.5208089, 0.10708917]

Real: [0.14544241, 0.54794455, 0.058469053]
Generated: [0.15082234, 0.5798031, 0.124466926]

Real: [0.19421951, 0.5763029, 0.17130229]
Generated: [0.11141093, 0.54463303, 0.16677524]

Real: [-0.1332553, 0.25359568, 0.15247762]
Generated: [-0.0802687, 0.25949344, 0.19595703]

Real: [-0.02517629, 0.4329522, -0.0010937594]
Generated: [0.012502683, 0.46983865, 0.0010313274]

Real: [-0.24630585, 0.37629125, 0.10068678]
Generated: [-0.17809457, 0.3451189, -0.031150872]

Real: [0.34586105, 0.45578426, 0.40945578]
Generated: [0.37359056, 0.3520332, 0.2864734]

Real: [0.34750